In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
import os
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data_dir = '/kaggle/input/task-1-data/data'
image_size = 128
data_transform = transforms.Compose([
    transforms.Resize([image_size, image_size]),
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

full_dataset = datasets.ImageFolder(root=data_dir, transform=data_transform)

train_size = int(0.8* len(full_dataset))
val_size = (len(full_dataset) - train_size)//2
test_size = len(full_dataset) - train_size -  val_size

train_dataset, val_dataset, test_dataset = random_split(full_dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader  = DataLoader(val_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)


class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, padding=1, kernel_size=3),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(kernel_size=2),
            
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=2),
            
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(kernel_size=2),
        )
        
        self.dense_layers = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(0.3),
            nn.Linear(128 * 16 * 16, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.3),
            nn.Linear(256, 100)
        )
    
    def forward(self,x):
        x =self.features(x)
        x = self.dense_layers(x)
        return x
    
model = Classifier().to(device)

loss_func = nn.CrossEntropyLoss()
optimiser = optim.Adam(model.parameters(), lr=0.0003)

num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0
    correct = 0
    total = 0
    
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        loss = loss_func(outputs, labels)
        
        optimiser.zero_grad()
        loss.backward()
        optimiser.step()
        
        running_loss += loss.item()
        max_val, prediction = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (prediction==labels).sum().item()
        
    train_loss = running_loss/len(train_loader)
    train_accuracy = (correct/total)*100
    
    
    model.eval()
    val_loss = 0
    correct = 0
    total = 0
    
    for images, labels in val_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        loss = loss_func(outputs, labels)
        
        val_loss += loss.item()
        max_val, prediction = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (prediction == labels).sum().item()
        
    val_loss = val_loss / len(val_loader)
    val_acc = 100 * correct / total
        
    print(f'Epoch [{epoch+1}/{num_epochs}]')
    print(f'Train Loss: {train_loss:.4f}, Acc: {train_accuracy:.2f}%')
    print(f'Val Loss: {val_loss:.4f}, Acc: {val_acc:.2f}%')

Epoch [1/10]
Train Loss: 4.2202, Acc: 8.01%
Val Loss: 3.6517, Acc: 22.27%
Epoch [2/10]
Train Loss: 2.9709, Acc: 53.41%
Val Loss: 2.6508, Acc: 63.18%
Epoch [3/10]
Train Loss: 1.7254, Acc: 91.22%
Val Loss: 2.0025, Acc: 78.64%
Epoch [4/10]
Train Loss: 0.8858, Acc: 98.92%
Val Loss: 1.7800, Acc: 81.36%
Epoch [5/10]
Train Loss: 0.4476, Acc: 99.91%
Val Loss: 1.4750, Acc: 85.23%
Epoch [6/10]
Train Loss: 0.2139, Acc: 100.00%
Val Loss: 1.3199, Acc: 88.64%
Epoch [7/10]
Train Loss: 0.1211, Acc: 100.00%
Val Loss: 1.2815, Acc: 86.59%
Epoch [8/10]
Train Loss: 0.0776, Acc: 100.00%
Val Loss: 1.2350, Acc: 87.27%
Epoch [9/10]
Train Loss: 0.0578, Acc: 100.00%
Val Loss: 1.1301, Acc: 89.09%
Epoch [10/10]
Train Loss: 0.0402, Acc: 100.00%
Val Loss: 1.0762, Acc: 88.64%


In [3]:
model.eval()
test_loss = 0
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        loss = loss_func(outputs, labels)
        
        test_loss += loss.item()
        max_val, prediction = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (prediction == labels).sum().item()
        
test_loss = test_loss / len(test_loader)
test_acc = 100 * correct / total

print(f'\nTest Results:')
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.2f}%')


Test Results:
Test Loss: 1.0911, Test Accuracy: 88.18%
